In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import project_func as pf
import os
import glob
import pandas as pd
import numpy as np

In [3]:
def extract_historical(df, search_cat, search_for):
    if search_cat == 'vintage':
        code_locator = 0
    elif search_cat == 'cz':
        code_locator = 1
    elif search_cat == 'scen_year':
        code_locator = 2
    elif search_cat == 'bldg':
        code_locator = 3
    else:
        print('search_for arg must in vintage, cz, scen_year, or bldg')
    final_idx = []
    null_idx = []
    for idx in list(df.index):
        if idx.split("_")[code_locator]==str(search_for):
            final_idx.append(idx)
        else:
            null_idx.append(idx)
            
    return final_idx, null_idx

def reduce_key_scen_year(search_key):
    return "_".join([i for n, i in enumerate(search_key.split("_")) if n!=2])

def find_matching_key(search_key, search_set):
    search_set_reduced = [reduce_key_scen_year(k) for k in search_set]
    search_set = list(zip(search_set_reduced, search_set))

    # def find_match(search_key, search_set):
    reduced_key = reduce_key_scen_year(search_key)

    search_match = [k for r, k in search_set if r==reduced_key]
    return search_match

# Annual Results

# Seasonal Results

In [4]:
seasonal_file = "/Users/jmccarty/GitHub/cmip6_and_buildings/notebooks/output_data/seasonal/Spring.csv"
df = pd.read_csv(seasonal_file, index_col='Unnamed: 0').rename(columns=pf.reverse_dict(pf.map_perf_metrics))

hist_idx, null_idx = extract_historical(df, 'scen_year', 0)
hist_df = df.loc[hist_idx]
fut_df = df.loc[null_idx]
fut_df

,hs_el_kwh,hs_ng_kwh,cs_el_kwh,fan_el_kwh,fac_ng_kwh,fac_el_kwh,geff_irrad_kwh,cell_temp_degc,pv_yield_kwh,fac_el_net_kwh,pv_consumed_kwh,pv_excess_kwh,self_suff_pct,self_consume_pct,op_em_kgco2,op_em_pv_kgco2,mit_pot_kgco2
2_0_1_4,350.20,0.00,71108.89,6630.02,1623.75,212491.43,385601.70,31.63,102315.21,121279.66,91211.77,11103.44,42.92,89.15,71773.01,41036.45,30736.56
2_0_1_5,0.00,10.19,32562.74,6559.50,11517.16,88293.41,188290.07,28.83,49132.63,56286.46,32006.96,17125.67,36.25,65.14,31880.85,21033.06,10847.78
2_0_2_4,310.25,0.00,74015.36,6745.00,1623.75,215472.92,386838.33,31.96,102486.65,123754.28,91718.64,10768.01,42.57,89.49,72782.82,41870.19,30912.63
2_0_2_5,0.00,7.42,34145.00,6828.02,11514.39,90144.20,188933.92,29.16,49248.14,57587.55,32556.65,16691.49,36.12,66.11,32496.35,21464.76,11031.59
2_0_3_4,271.05,0.00,73924.70,6746.43,1623.75,215344.50,385400.89,31.97,102131.88,123880.89,91463.61,10668.27,42.47,89.55,72715.54,41891.80,30823.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2_15_10_5,8.38,50046.23,1338.81,2398.66,80419.90,51817.05,128232.05,4.86,32685.89,34097.14,17719.91,14965.98,34.20,54.21,34682.65,28656.39,6026.26
2_15_11_4,40353.07,31754.09,3446.65,4143.27,35162.03,179305.55,272240.04,7.08,77392.12,114501.13,64804.41,12587.70,36.14,83.74,76473.02,54531.20,21941.82
2_15_11_5,8.30,49069.82,1439.93,2401.36,79443.49,51920.79,128995.51,5.05,32932.05,34079.65,17841.14,15090.92,34.36,54.18,34537.86,28467.78,6070.08
2_15_12_4,39142.74,30551.21,3836.41,4134.36,33959.15,178476.07,273102.91,7.49,77552.91,113463.78,65012.29,12540.63,36.43,83.83,75857.44,53835.65,22021.79


In [9]:

delta_dict = {}
col = fut_df.columns[0]
delta_dict[col] = {}
for idx_key in fut_df.index:
    match_hist_key = find_matching_key(idx_key, hist_df.index.tolist())
    col_delta = fut_df.loc[idx_key][col] - hist_df.loc[match_hist_key][col].values[0]
    delta_dict[col].update({idx_key:col_delta})

In [32]:
delta_df = pd.DataFrame(delta_dict)
delta_df

,hs_el_kwh
2_0_10_4,-439.78
2_0_10_5,0.00
2_0_11_4,-546.67
2_0_11_5,0.00
2_0_12_4,-630.04
...,...
2_9_7_5,0.00
2_9_8_4,-3555.84
2_9_8_5,0.00
2_9_9_4,-2381.77


In [44]:
pf.reverse_sim_code(pf.all_dict,'2_9_7_5')

'new_4c_ssp245_2065_RefBldgMidriseApartment'

In [43]:
pf.all_dict['scen_year_r']

{0: 'historical_2020',
 1: 'ssp126_2035',
 2: 'ssp126_2050',
 3: 'ssp126_2065',
 4: 'ssp126_2080',
 5: 'ssp245_2035',
 6: 'ssp245_2050',
 7: 'ssp245_2065',
 8: 'ssp245_2080',
 9: 'ssp585_2035',
 10: 'ssp585_2050',
 11: 'ssp585_2065',
 12: 'ssp585_2080'}

In [45]:
delta_df.join(pf.expand_code_idx_to_df(delta_df, pf.all_dict, pf.code_names))

,hs_el_kwh,vintage,climate_zone,scenario,year,bldg_type
2_0_10_4,-439.78,new,1a,ssp585,2050,RefBldgMediumOffice
2_0_10_5,0.00,new,1a,ssp585,2050,RefBldgMidriseApartment
2_0_11_4,-546.67,new,1a,ssp585,2065,RefBldgMediumOffice
2_0_11_5,0.00,new,1a,ssp585,2065,RefBldgMidriseApartment
2_0_12_4,-630.04,new,1a,ssp585,2080,RefBldgMediumOffice
...,...,...,...,...,...,...
2_9_7_5,0.00,new,4c,ssp245,2065,RefBldgMidriseApartment
2_9_8_4,-3555.84,new,4c,ssp245,2080,RefBldgMediumOffice
2_9_8_5,0.00,new,4c,ssp245,2080,RefBldgMidriseApartment
2_9_9_4,-2381.77,new,4c,ssp585,2035,RefBldgMediumOffice


{0: 'vintage', 1: 'climate_zone', 2: 'scenario', 3: 'year', 4: 'bldg_type'}